# Task 3: Building Summary Tables

#### You will submit your Python code that builds the database. 

Here I connect to Google Big Query to the stored clean files on the umt-msba databse to the transactions project. 


## Connecting to GBQ

In [5]:
import os
import re
import datetime
import sqlite3


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
from sqlalchemy import create_engine

In [21]:
#Set Paths for GBQ
service_path = "/Users/meganalbee/Desktop/ADA/key/"
service_file = 'albee-msba-4037d70faf07.json' # My Key   
gbq_proj_id = 'albee-msba' # My GBQ 
dataset_id = 'wedge_msba' #Set to the Wedge

#Private Key. Do not change. 
private_key = service_path + service_file

In [7]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [8]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## Queries for Task
There are three total. 

1. Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items .

2. Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.

3. Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items. 

## Query 1
Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items .

In [29]:
query1 = """

        SELECT EXTRACT(date FROM datetime) as date,
        EXTRACT(HOUR FROM datetime) as hour,
        SUM(total) as tot_spend,
        COUNT(DISTINCT Concat(
            cast(EXTRACT(date FROM datetime) as string), 
            cast(register_no as string), 
            cast(emp_no as string), 
            (cast(trans_no as string)))) as no_trans, 
        SUM(CASE WHEN trans_status in ('V','R') THEN -1
            ELSE 1 
            END) AS no_items
FROM `albee-msba.wedge_msba.transArchive_*`
WHERE department != '0'
AND department != '15'
AND (trans_status is NULL or
     trans_status in (' ', 'V','R'))
GROUP BY date, hour
ORDER BY date, hour

"""


sales_date_hour = pandas_gbq.read_gbq(query1,project_id = gbq_proj_id)


Downloading: 100%|███████████████████| 39334/39334 [00:02<00:00, 15073.28rows/s]


In [30]:
sales_date_hour

,date,hour,tot_spend,no_trans,no_items
0,2010-01-01,9,1003.29,36,244
1,2010-01-01,10,3128.55,82,913
2,2010-01-01,11,4001.66,118,1108
3,2010-01-01,12,3886.51,124,1143
4,2010-01-01,13,4654.52,154,1365
...,...,...,...,...,...
39329,2017-01-31,18,6992.52,255,1713
39330,2017-01-31,19,5386.16,208,1378
39331,2017-01-31,20,4635.16,160,1236
39332,2017-01-31,21,2910.39,115,833


## Query 2
Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.

In [27]:
query2 = """
            SELECT card_no, 
        EXTRACT(YEAR FROM datetime) as year,
        EXTRACT(MONTH FROM datetime) as month,
        SUM(total) as tot_spend,
        COUNT(DISTINCT Concat(
            cast(EXTRACT(date FROM datetime) as string), 
            cast(register_no as string), 
            cast(emp_no as string), 
            (cast(trans_no as string)))) as no_trans, 
        SUM(CASE WHEN trans_status in ('V','R') THEN -1
            ELSE 1 
            END) AS no_items
        FROM `albee-msba.wedge_msba.transArchive_*`
        WHERE department != '0'
        AND department != '15'
        AND (trans_status is NULL or
             trans_status in (' ', 'V','R'))
        GROUP BY card_no, year, month
        ORDER BY year, month

"""
sales_by_owner_ym = pandas_gbq.read_gbq(query2,project_id = gbq_proj_id)


Downloading: 100%|█████████████████| 808964/808964 [00:41<00:00, 19675.53rows/s]


In [31]:
sales_by_owner_ym

,card_no,year,month,tot_spend,no_trans,no_items
0,50449,2010,1,513.41,5,179
1,14022,2010,1,483.63,3,107
2,16606,2010,1,916.61,4,211
3,12225,2010,1,131.86,2,59
4,30454,2010,1,371.76,7,102
...,...,...,...,...,...,...
808959,12262,2017,1,66.62,2,22
808960,20083,2017,1,62.79,1,17
808961,23088,2017,1,12.38,1,3
808962,10757,2017,1,1.79,1,1


## Query 3
Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.


In [15]:
df = pd.read_csv('department_lookup.csv')

In [23]:
df['department'] = df['department'].astype('str')

In [24]:
# Your code here
table_name = 'department_lookup'
table_id = ".".join([gbq_proj_id,dataset_id,table_name])
pandas_gbq.to_gbq(df, table_id, project_id=gbq_proj_id,if_exists="replace") # beam it up scotty

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 12018.06it/s]


In [32]:
query3 = """
            SELECT upc,
                transacts.description as prod_descript,
                transacts.department as departments,
                deptlu.dept_name as dept_name,
                EXTRACT(YEAR FROM transacts.datetime) as year,
                EXTRACT(MONTH FROM transacts.datetime) as month,
                SUM(transacts.total) as tot_spend,
                  COUNT(DISTINCT Concat(
                  cast(EXTRACT(date FROM datetime) as string), 
                  cast(register_no as string), 
                  cast(emp_no as string), 
                  (cast(trans_no as string)))) as no_trans, 
                SUM(CASE WHEN trans_status in ('V','R') THEN -1
                  ELSE 1 
                    END) AS no_items
              FROM `albee-msba.wedge_msba.transArchive_*` transacts
               LEFT JOIN `albee-msba.wedge_msba.department_lookup` deptlu 
                 USING (department)
              WHERE transacts.department != '0'
                  AND transacts.department != '15'
                  AND (trans_status is NULL or
                    trans_status in (' ', 'V','R'))
              GROUP BY prod_descript, year, month, upc, departments, deptlu.dept_name
              ORDER BY tot_spend DESC;

"""
sales_by_prod_year_month = pandas_gbq.read_gbq(query3,project_id = gbq_proj_id)


Downloading: 100%|███████████████| 1185698/1185698 [01:16<00:00, 15401.60rows/s]


In [33]:
sales_by_prod_year_month

,upc,prod_descript,departments,dept_name,year,month,tot_spend,no_trans,no_items
0,0,Change,0.0,None,2015,3,174023.01,86686,86948
1,0,Change,0.0,None,2015,2,161915.29,71809,72010
2,0,Change,0.0,None,2015,1,156136.09,72168,72448
3,0,Change,0.0,None,2015,10,151227.12,82567,82852
4,TAX,Tax,0.0,None,2015,3,51388.53,86686,86948
...,...,...,...,...,...,...,...,...,...
1185693,0,Cash,0.0,None,2015,3,-559807.45,23755,23828
1185694,0,Credit Card,0.0,None,2015,2,-2054993.21,50771,51404
1185695,0,Credit Card,0.0,None,2015,10,-2085196.04,57698,58363
1185696,0,Credit Card,0.0,None,2015,1,-2230659.52,51444,52186


## Filling DB
Here we will populate our SQLite3 data base with our tables by Query per instructions. 

In [36]:
db_wedge = sqlite3.connect("wedge_summary.db")

cur = db_wedge.cursor()

In [37]:
#table to db
sales_date_hour.to_sql(name='query1',con = db_wedge)

39334

In [38]:
sales_by_owner_ym.to_sql(name='query2',con = db_wedge)

808964

In [39]:
sales_by_prod_year_month.to_sql(name='query3',con = db_wedge)

1185698

In [40]:
#close connection
cur.close() 

In [42]:
#db close here 
db_wedge.close() 